# Введение в обработку текста на естественном языке

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Введение в обработку текста на естественном языке"
* https://www.nltk.org/api/nltk.metrics.distance.html
* https://pymorphy2.readthedocs.io/en/stable/user/guide.html
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [1]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 46.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=a4ab8175b7a04792c7402663f0bc6e60e31873c045a4ea781912c9ba7a385657
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [19]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. При помощи расстояния Левенштейна иправьте опечатку в слове "велечайшим".

In [20]:
from nltk import edit_distance

2. Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов.

In [4]:
from nltk.stem import SnowballStemmer
from nltk import word_tokenize
import pymorphy2

In [ ]:
text = '''Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов.'''

3. Преобразуйте предложения из формулировки задания 2 в векторы при помощи `CountVectorizer`. Выведите на экран словарь обученного токенизатора.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import sent_tokenize

In [ ]:
text = '''Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов.'''

## Лабораторная работа 6

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy` и `pandas`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy` или структур `pandas` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import random
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.metrics import edit_distance
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
import pymorphy2
from nltk.corpus import stopwords
nltk.download('stopwords')
from collections import Counter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


1\. Загрузите данные из файла `ru_recipes_sample.csv` в виде `pd.DataFrame` `recipes` Используя регулярные выражения, удалите из описаний (столбец `description`) все символы, кроме русских букв, цифр и пробелов. Приведите все слова в описании к нижнему регистру. Сохраните полученный результат в столбец `description`.

In [43]:
def delete_sumbol(df):
  df['description'] = df['description'].str.lower()
  df['description'] = df['description'].apply(lambda x: re.sub('[^а-яё0-9\s]', '', x))
  return df

recipes_sample = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/data2/ru_recipes_sample.csv')
recipes = delete_sumbol(recipes_sample)
recipes.head(10)

,url,name,ingredients,description
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах...",этот коктейль готовлю из замороженной клубники...
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку...",быстро и вкусно
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '...",сытный овощной салатик пальчики оближете
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'Фарш куриный': '800 г', 'Пюре картофельное':...",картофельное пюре и куриные котлеты вкусная к...
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар': '30...",вишневая наливка имеет яркий вишневый вкус кот...
5,https://www.povarenok.ru/recipes/show/167765/,Песочный пирог с тыквенным суфле,"{'Масло сливочное': '100 г', 'Сахар': '50 г', ...",ароматное тыквенное суфле на песочном тесте пр...
6,https://www.povarenok.ru/recipes/show/100230/,Шоколадные конфеты ручной работы,"{'Какао-масло': '100 г', 'Какао тертое': '200 ...",конечно купить в магазине готовые шоколадные к...
7,https://www.povarenok.ru/recipes/show/96257/,Рыбно-тыквенный гратен,"{'Масло растительное': '3 ст. л.', 'Рыба': '40...",этот рецепт горячей закуски чем то отдаленно н...
8,https://www.povarenok.ru/recipes/show/139360/,"Плов с креветками из риса ""Басмати""","{'Рис': '2 стак.', 'Вода': '4 стак.', 'Морковь...",плов с креветками из риса очень вкусный и пол...
9,https://www.povarenok.ru/recipes/show/96774/,Молочное мороженое,"{'Молоко': '450 г', 'Желток яичный': '3 шт', '...",рецепт вкусного мороженого из молока подробнее...


### Расстояние редактирования

2\. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние Левенштейна. Выведите на экран результат в следующем виде:

```
d(word1, word2) = x
```

In [36]:
nltk.download('punkt')

reg = r"[а-яА-Я]+"
tokenizer = RegexpTokenizer(reg)
words = set(tokenizer.tokenize(str(recipes['description'])))
word = []
for i in range(5):
  a = random.sample(words, 2)
  word.append(a)

for i in word:
  distance = edit_distance(i[0], i[1])
  print(f"d({i[0]}, {i[1]}) = {distance}")

{'готовлю', 'фантазии', 'продолжаются', 'но', 'тех', 'из', 'кто', 'очень', 'мое', 'клубники', 'вам', 'одуванчиков', 'оближете', 'варенье', 'блюдо', 'варила', 'овощной', 'прошлым', 'пальчики', 'для', 'заказ', 'только', 'имеет', 'замороженной', 'сюда', 'кот', 'салатик', 'корочки', 'любит', 'в', 'этот', 'летом', 'сделал', 'любимое', 'предлагаю', 'баклажановые', 'лазанья', 'по', 'фар', 'картофельное', 'хлеба', 'наливка', 'вкусная', 'пюре', 'сытный', 'котлеты', 'вишневый', 'и', 'куриные', 'к', 'буратино', 'коктейль', 'быстро', 'яркий', 'вкусно', 'вишневая', 'чечевицу', 'вкус', 'три', 'кушать'}
d(имеет, пюре) = 4
d(сытный, по) = 6
d(предлагаю, вишневый) = 9
d(овощной, оближете) = 7
d(по, готовлю) = 6


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-36-159a69d60322>:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  a = random.sample(words, 2)


3\. Напишите функцию, которая принимает на вход 2 текстовые строки `s1` и `s2` и строит матрицу попарных расстояний. Функция должна возвращать `pd.DataFrame`, в котором названий столбцов и строк соответствуют словам из `s1` и `s2`. 
Продемонстрируйте работу вашей функции на примере описаний двух рецептов с ID 135488 и 851934 (ID рецепта - это число, стоящее в конце url рецепта). Выведите на экран описания этих рецептов и результат работы функции.

In [55]:
def is_plagiarism(s1: str, s2: str) -> pd.DataFrame:
  str1 = s1.split()
  str2 = s2.split()
  df = pd.DataFrame(index=str1, columns=str2)
  for i in str1:
    for j in str2:
      df.loc[i, j] = edit_distance(i, j)
  return df


In [46]:
def test(x,reg):
  a=re.match(reg, str(x))
  if a!=None:
    return x

In [70]:
a = recipes.index[recipes['url']==recipes['url'].apply(test, args=(r'http[s]?://.*/135488',))]
b = recipes.index[recipes['url']==recipes['url'].apply(test, args=(r'http[s]?://.*/851934',))]
s1 = recipes.iloc[a]['description'].values
s2 = recipes.iloc[b]['description'].values
print(s1)
print(s2)
df = is_plagiarism(s1[0], s2[0])
df

['прекрасной закуской к крепким напиткам на фуршетном столе станет паштет из сала и авокадо с чесноком который мы спрячем в орешках из бородинского хлеба положив в середину маслину закуска получается необычной оригинальной и ценится особенно мужчинами']
['замечательной закуской к напиткам на фуршетном столе станет паштет из сала и авокадо с чесноком куда мы добавим орешков из бородинского хлеба положив в середину маслины закуска получается крайне необычной оригинальной и ценится особенно мужчинами']


,замечательной,закуской,к,напиткам,на,фуршетном,столе,станет,паштет,из,...,маслины,закуска,получается,крайне,необычной,оригинальной,и,ценится,особенно,мужчинами
прекрасной,9,6,9,9,9,8,9,8,8,10,...,9,8,8,6,6,7,10,8,8,10
закуской,9,0,7,6,7,8,7,8,7,8,...,7,2,9,7,7,10,8,8,8,9
к,13,7,0,7,2,9,5,6,6,2,...,7,6,10,5,9,12,1,7,8,9
крепким,12,7,6,6,7,6,7,7,7,6,...,7,7,10,5,9,11,6,7,8,8
напиткам,11,6,7,0,6,7,7,7,6,7,...,7,5,10,8,8,11,7,6,8,7
на,12,7,2,6,0,8,5,5,5,2,...,6,6,9,5,8,10,2,6,7,7
фуршетном,10,8,9,7,8,0,8,7,7,9,...,8,9,8,7,7,10,9,8,6,7
столе,11,7,5,7,5,8,0,3,5,5,...,6,7,8,5,8,11,5,7,6,9
станет,11,8,6,7,5,7,3,0,4,6,...,6,7,8,4,8,10,6,6,6,8
паштет,10,7,6,6,5,7,5,4,0,6,...,6,6,7,5,9,11,6,6,7,9


### Стемминг, лемматизация

4\. На основе набора слов из задания 2 создайте `pd.DataFrame` со столбцами `word`, `stemmed_word` и `normalized_word`. В столбец `stemmed_word` поместите версию слова после проведения процедуры стемминга; в столбец `normalized_word` поместите версию слова после проведения процедуры лемматизации. Столбец `word` укажите в качестве индекса. 

Для стемминга можно воспользоваться `SnowballStemmer` из `nltk`, для лемматизации слов - пакетом `pymorphy2`. Сравните результаты стемминга и лемматизации. Поясните на примере одной из строк получившегося фрейма (в виде текстового комментария), в чем разница между двумя этими подходами. 

In [73]:
import warnings
warnings.filterwarnings('ignore')

In [80]:
def lem(lemm):
  df = pd.DataFrame(columns=['word', 'stemmed_word', 'normalized_word'])
  for word in words:
    stemmed_word = ps.stem(word)
    normalized_word = lemm.parse(word)[0].normal_form
    df = df.append({'word': word, 'stemmed_word': stemmed_word, 'normalized_word': normalized_word}, ignore_index=True)
  return df.set_index('word')

In [81]:
ps = SnowballStemmer(language="russian")
lemmatizer = pymorphy2.MorphAnalyzer()
print(lem(lemmatizer))

             stemmed_word normalized_word
word                                     
готовлю            готовл        готовить
фантазии           фантаз        фантазия
продолжаются     продолжа    продолжаться
но                     но              но
тех                   тех             тот
из                     из              из
кто                   кто             кто
очень                очен           очень
мое                    мо             мой
клубники          клубник        клубника
вам                   вам              вы
одуванчиков     одуванчик       одуванчик
оближете          оближет        облизать
варенье             варен         варение
блюдо                блюд           блюдо
варила                вар          варить
овощной             овощн         овощной
прошлым             прошл         прошлое
пальчики          пальчик         пальчик
для                   для             для
заказ               заказ           заказ
только              тольк         

5\. Добавьте в таблицу `recipes` столбец `description_no_stopwords`, в котором содержится текст описания рецепта после удаления из него стоп-слов. Посчитайте и выведите на экран долю стоп-слов среди общего количества слов (включая возможные дубликаты). Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [ ]:
stopwords.words('russian')

In [104]:
def des(recipes):
  stopword = set(stopwords.words('russian'))
  recipes['description_no_stopwords'] = ''
  for index, row in recipes.iterrows():
     words_no_stopwords = [word for word in word_tokenize(row['description']) if word not in stopword]
     recipes.at[index, 'description_no_stopwords'] = ' '.join(words_no_stopwords)

  words=recipes['description'].apply(word_tokenize)
  all_words=sum(words,[])
  no_stops = [word for word in all_words if word not in stopword]
  ratio = len(all_words) / len(no_stops)
  print(r'Доля стоп-слов среди общего количества слов: ', ratio)
  words_before = Counter(all_words).most_common(10)
  words_after = Counter(no_stops).most_common(10)
  print('Топ-10 самых часто употребляемых слов до: ', words_before)
  print('Топ-10 самых часто употребляемых слов после: ', words_after)
  return recipes

a = des(recipes)
a

Доля стоп-слов среди общего количества слов:  1.4762812701598451
Топ-10 самых часто употребляемых слов до:  [('и', 5054), ('в', 2584), ('с', 1934), ('на', 1655), ('очень', 1607), ('не', 1517), ('из', 1006), ('я', 979), ('рецепт', 869), ('а', 863)]
Топ-10 самых часто употребляемых слов после:  [('очень', 1607), ('рецепт', 869), ('это', 734), ('блюдо', 524), ('вкусный', 461), ('просто', 436), ('вкусно', 375), ('приготовить', 344), ('вкус', 324), ('салат', 313)]


,url,name,ingredients,description,description_no_stopwords
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах...",этот коктейль готовлю из замороженной клубники...,коктейль готовлю замороженной клубники клубник...
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку...",быстро и вкусно,быстро вкусно
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '...",сытный овощной салатик пальчики оближете,сытный овощной салатик пальчики оближете
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'Фарш куриный': '800 г', 'Пюре картофельное':...",картофельное пюре и куриные котлеты вкусная к...,картофельное пюре куриные котлеты вкусная клас...
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар': '30...",вишневая наливка имеет яркий вишневый вкус кот...,вишневая наливка имеет яркий вишневый вкус кот...
...,...,...,...,...,...
3462,https://www.povarenok.ru/recipes/show/54574/,Мшош,"{'Чечевица': '1 стак.', 'Лук репчатый': '2 шт'...",для тех кто любит чечевицу вам сюда очень вкус...,тех любит чечевицу сюда очень вкусная чечевичн...
3463,https://www.povarenok.ru/recipes/show/113494/,Мясные треугольники с баклажаном,"{'Фарш мясной': '400 г', 'Баклажан': '1 шт', '...",баклажановые фантазии продолжаются предлагаю в...,баклажановые фантазии продолжаются предлагаю в...
3464,https://www.povarenok.ru/recipes/show/83228/,"""Болоньез"" по-новому","{'Фарш мясной': '400 г', 'Томаты в собственном...",мое любимое блюдо лазанья но кушать только фар...,мое любимое блюдо лазанья кушать фарш поднадое...
3465,https://www.povarenok.ru/recipes/show/172238/,Варенье из одуванчиков с апельсинами,"{'Цветки': '400 г', 'Сахар': '1300 г', 'Апельс...",прошлым летом варила варенье из одуванчиков по...,прошлым летом варила варенье одуванчиков рецеп...


### Векторное представление текста

6\. Выберите случайным образом 5 рецептов из набора данных, в названии которых есть слово "оладьи" (без учета регистра). Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`. На основе полученных векторов создайте `pd.DataFrame`, в котором названия колонок соответствуют словам из словаря объекта-векторизатора. 

Примечание: обратите внимание на порядок слов при создании колонок.

In [ ]:
recipes.head(2)

,url,name,ingredients,description
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах...",этот коктейль готовлю из замороженной клубники...
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку...",быстро и вкусно


In [107]:
recipes1 = recipes[recipes['name'].str.contains(r"\bоладьи")]
random_recipes = recipes1.sample(n=5, replace=False)
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(random_recipes['description'])

df = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names_out())
df

,200мл,500,без,блины,блюдо,больше,бутерброд,был,быстрее,быстро,...,хозяек,холодными,хороши,хороший,хорошо,что,шалот,это,этом,этому
0,0.000000,0.000000,0.15299,0.15299,0.123431,0.000000,0.000000,0.000000,0.15299,0.15299,...,0.15299,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.123431,0.000000,0.000000
1,0.141043,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.113793,0.00000,0.00000,...,0.00000,0.000000,0.000000,0.141043,0.141043,0.141043,0.141043,0.000000,0.141043,0.000000
2,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.130278,0.00000,0.00000,0.105107,0.130278,0.130278,0.105107,0.00000,0.00000,...,0.00000,0.130278,0.130278,0.000000,0.000000,0.000000,0.000000,0.105107,0.000000,0.130278
4,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


7\. Вычислите схожесть между каждой парой рецептов, используя полученные в задании 6 векторные представления и функцию косинусного расстояния (можно воспользоваться функциями из любого пакета: `scipy`, `scikit-learn` или реализовать функцию самому). Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов. При расчете косинусного расстояния для всех пар не допускается использование циклов.

Примечание: обратите внимание, что $d_{cosine}(x, x) = 0$

In [108]:
vectors_cos = vectors * vectors.T
vec = pd.DataFrame(1-(vectors_cos.toarray()).round(4), columns=random_recipes['name'])
vec = vec.set_index(random_recipes['name'])
vec

name,Воздушные оладьи,Овсяные оладьи с луково-гороховой начинкой,Тыквенные оладьи,Куриные оладьи с сыром,Картофельно-творожные оладьи
name,,,,,
Воздушные оладьи,0.0000,0.9370,0.9153,0.9208,0.9178
Овсяные оладьи с луково-гороховой начинкой,0.9370,0.0000,0.9531,0.9105,0.9102
Тыквенные оладьи,0.9153,0.9531,0.0000,0.9567,0.9752
Куриные оладьи с сыром,0.9208,0.9105,0.9567,0.0000,0.9559
Картофельно-творожные оладьи,0.9178,0.9102,0.9752,0.9559,0.0000


8\. Напишите функцию, которая принимает на вход `pd.DataFrame`, полученный в задании 7, и возвращает в виде кортежа названия двух различных рецептов, которые являются наиболее похожими. Прокомментируйте результат (в виде текстового комментария). Для объяснения результата сравните слова в описаниях двух этих отзывов.

In [109]:
def find_closest(sim_df: pd.DataFrame) -> tuple:
  sim_df[sim_df[:]==0] = np.nan
  minn = sim_df.min(skipna=True).min()
  check_na = sim_df[sim_df.isin([minn])].notna()
  c = [(sim_df.index[i], sim_df.columns[c]) for i, c in zip(*np.where(sim_df.isin([minn])))]
  return c[0]


In [110]:
rez = find_closest(vec)
print(rez)
print(random_recipes[random_recipes['name'] == rez[0]]['description'].values)
print(random_recipes[random_recipes['name'] == rez[1]]['description'].values)

('Овсяные оладьи с луково-гороховой начинкой', 'Картофельно-творожные оладьи')
['очень вкусные нежный оладьи несмотря на то что состоят из нескольких ингредиентов начинка из лука с зеленым горошком и небольшим колвом фарша хорошо дополняют вкус подавать можно со сметаной стакан у меня 200мл лук шалот можно заменить на обычный просто у меня его в этом году был хороший урожай вот и кладу его везде угошайтесь']
['картофельное пюре оставшееся с ужина утром можно превратить в сытный и полезный завтрак дополнив его еще несколькими ингредиентами а именно в картофельнотворожные оладьи которые получаются нежными необычными и очень вкусными']
